#### <code>Hough Line Transform</code>
The Hough Transform is a popular technique to detect any shape, if you can represent that shape in a mathematical form. It can detect the shape even if it is broken or distorted a little bit.
* The Hough Line Transform is a transform used to detect straight lines.
* To apply the Transform, first an edge detection pre-processing is desirable.

#### <code>How does it work?</code>
As you know, a line in the image space can be expressed with two variables. For example:
* In the Cartesian coordinate system: Parameters: (m,b).
* In the Polar coordinate system: Parameters: (r,θ)

<img src="https://docs.opencv.org/master/Hough_Lines_Tutorial_Theory_0.jpg" alt="drawing" width="250"/>

For Hough Transforms, we will express lines in the Polar system. Hence, a line equation can be written as:

$y = \left ( -\dfrac{\cos \theta}{\sin \theta} \right ) x + \left ( \dfrac{r}{\sin \theta} \right )$

Arranging the terms: r=xcosθ+ysinθ

1. In general for each point (x0,y0), we can define the family of lines that goes through that point as:

$r_{\theta} = x_{0} \cdot \cos \theta + y_{0} \cdot \sin \theta$

Meaning that each pair (rθ,θ) represents each line that passes by (x0,y0).

2. If for a given (x0,y0) we plot the family of lines that goes through it, we get a sinusoid. For instance, for x0=8 and y0=6 we get the following plot (in a plane θ - r):


<img src="https://docs.opencv.org/master/Hough_Lines_Tutorial_Theory_1.jpg" alt="drawing" width="300"/>
We consider only points such that r>0 and 0<θ<2π.

3. We can do the same operation above for all the points in an image. If the curves of two different points intersect in the plane θ - r, that means that both points belong to a same line. For instance, following with the example above and drawing the plot for two more points: x1=4, y1=9 and x2=12, y2=3, we get:

<img src="https://docs.opencv.org/master/Hough_Lines_Tutorial_Theory_2.jpg" alt="drawing" width="300"/>
The three plots intersect in one single point (0.925,9.6), these coordinates are the parameters ( θ,r) or the line in which (x0,y0), (x1,y1) and (x2,y2) lay.

4. What does all the stuff above mean? It means that in general, a line can be detected by finding the number of intersections between curves.The more curves intersecting means that the line represented by that intersection have more points. In general, we can define a threshold of the minimum number of intersections needed to detect a line.

5. This is what the Hough Line Transform does. It keeps track of the intersection between curves of every point in the image. If the number of intersections is above some threshold, then it declares it as a line with the parameters (θ,rθ) of the intersection point.
#### Standard and Probabilistic Hough Line Transform
OpenCV implements two kind of Hough Line Transforms:

#### <code>a. The Standard Hough Transform </code>

It consists in pretty much what we just explained in the previous section. It gives you as result a vector of couples (θ,rθ)
In OpenCV it is implemented with the function HoughLines()
#### <code>b. The Probabilistic Hough Line Transform<code>

A more efficient implementation of the Hough Line Transform. It gives as output the extremes of the detected lines (x0,y0,x1,y1)
In OpenCV it is implemented with the function HoughLinesP()

#### <code>Theory </code>

The Hough Transform is a popular technique to detect any shape, if you can represent that shape in a mathematical form. It can detect the shape even if it is broken or distorted a little bit. We will see how it works for a line.

A line can be represented as y=mx+c or in a parametric form, as ρ=xcosθ+ysinθ where ρ is the perpendicular distance from the origin to the line, and θ is the angle formed by this perpendicular line and the horizontal axis measured in counter-clockwise (That direction varies on how you represent the coordinate system. This representation is used in OpenCV). Check the image below:

<img src="Hough_transform.png" alt="drawing" width="400"/>

So if the line is passing below the origin, it will have a positive rho and an angle less than 180. If it is going above the origin, instead of taking an angle greater than 180, the angle is taken less than 180, and rho is taken negative. Any vertical line will have 0 degree and horizontal lines will have 90 degree.

Now let's see how the Hough Transform works for lines. Any line can be represented in these two terms, (ρ,θ). So first it creates a 2D array or accumulator (to hold the values of the two parameters) and it is set to 0 initially. Let rows denote the ρ and columns denote the θ. Size of array depends on the accuracy you need. Suppose you want the accuracy of angles to be 1 degree, you will need 180 columns. For ρ, the maximum distance possible is the diagonal length of the image. So taking one pixel accuracy, the number of rows can be the diagonal length of the image.

Consider a 100x100 image with a horizontal line at the middle. Take the first point of the line. You know its (x,y) values. Now in the line equation, put the values θ=0,1,2,....,180 and check the ρ you get. For every (ρ,θ) pair, you increment value by one in our accumulator in its corresponding (ρ,θ) cells. So now in accumulator, the cell (50,90) = 1 along with some other cells.

Now take the second point on the line. Do the same as above. Increment the values in the cells corresponding to (rho, theta) you got. This time, the cell (50,90) = 2. What you actually do is voting the (ρ,θ) values. You continue this process for every point on the line. At each point, the cell (50,90) will be incremented or voted up, while other cells may or may not be voted up. This way, at the end, the cell (50,90) will have maximum votes. So if you search the accumulator for maximum votes, you get the value (50,90) which says, there is a line in this image at a distance 50 from the origin and at angle 90 degrees. It is well shown in the below animation (Image Courtesy: Amos Storkey )


<img src="https://docs.opencv.org/master/houghlinesdemo.gif" alt="drawing" width="350"/>

#### cv.HoughLines (image, lines, rho, theta, threshold, srn = 0, stn = 0, min_theta = 0, max_theta = Math.PI)

* Everything explained above is encapsulated in the OpenCV function, cv.HoughLines(). It simply returns an array of :math:(rho, theta)` values. ρ is measured in pixels and θ is measured in radians.
* First parameter, Input image should be a binary image, so apply threshold or use canny edge detection before applying hough transform. Second and third parameters are ρ and θ accuracies respectively.
* Fourth argument is the threshold, which means the minimum vote it should get to be considered as a line. Remember, number of votes depends upon the number of points on the line. So it represents the minimum length of line that should be detected.

In [3]:

import cv2
import numpy as np

img = cv2.imread('C:/Users/piyus/All programming files/Computer_Vision/data/sudoku.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)
cv2.imshow('Canny_Edges', edges)

lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    # x1 stores the rounded off value of (r * cos(theta) - 1000 * sin(theta))
    x1 = int(x0 + 1000 * (-b))
    # y1 stores the rounded off value of (r * sin(theta)+ 1000 * cos(theta))
    y1 = int(y0 + 1000 * (a))
    # x2 stores the rounded off value of (r * cos(theta)+ 1000 * sin(theta))
    x2 = int(x0 - 1000 * (-b))
    # y2 stores the rounded off value of (r * sin(theta)- 1000 * cos(theta))
    y2 = int(y0 - 1000 * (a))
    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)


cv2.imshow('image', img)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

### Note:- As you can see that, we are getting infinite line on our image which can be avoided using HoughLinesP() method